# Second Home Assignment AA (2022/2023)
### Group: 59

Importar dados

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [2]:
biog = pd.read_csv("biodegradable_a.csv", sep=",")

X = biog.values[:, 0:41]
y = biog["Biodegradable"].values

y[y == "RB"] = 1.0
y[y == "NRB"] = 0.0

X_train_IVS, X_IVS, y_train_IVS, y_IVS = train_test_split(X, y.astype(float), test_size=0.33, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X_train_IVS, y_train_IVS, test_size=0.33, random_state=0)

X_cols = biog.columns[0:41]

countRB = 0
countNRB = 0
for i in y:
    if(i == 1): countRB += 1
    elif(i == 0): countNRB += 1

print(countRB, countNRB)

# pd.DataFrame(X, columns=X_cols)

3825 739


Verificar valores em falta

In [3]:
# check number of missing values in each column.

# dataset without lines with NaN values
biog_nan = biog.dropna()
#print(len(biog_nan))

print("No. of rows original dataset = {}".format(len(biog)))
print("No. of rows without NaN = {} ({}%)".format(len(biog_nan), round(len(biog_nan) / len(biog) * 100)))


# check number of missing values in each column.
missValues = biog.isna().sum()

missValues = missValues.to_frame()
print("Threshold number of missing values per column = {} (15%)".format(str(round(0.15 * len(X)))))

# print(missValues)

No. of rows original dataset = 4564
No. of rows without NaN = 889 (19%)
Threshold number of missing values per column = 685 (15%)


Escalar os dados (Standard Scaler)

In [4]:
import statistics

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Metodo de Imputação

In [5]:
useMean = [0, 1, 7, 11, 12, 13, 14, 21, 26, 27, 38]
useMedian = np.setdiff1d(range(41), useMean)

Imputar dados

In [6]:
# Impute missing values
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer 

# strategy = mean or median (for numerical data)
imputerMean = SimpleImputer(missing_values = np.nan, strategy = "mean") 
imputerMedian = SimpleImputer(missing_values = np.nan, strategy = "median") 

ct = ColumnTransformer(
    [('mean_imputer', imputerMean, useMean),
     ('median_imputer', imputerMedian, useMedian)]
)

ct.fit(X_train_scaled)

X_train_processed = ct.transform(X_train)
X_test_processed = ct.transform(X_test)

# Feature selection

Correlation

In [7]:
N, M = X_train.shape

v = np.hstack((y_train.reshape((N, 1)), X_train_processed))

pd.DataFrame(v)
corrMatrix = pd.DataFrame((np.corrcoef(v.T.astype(float))), columns=X_cols.insert(0, "Y")).iloc[0].sort_values()[0:10]

selectedCorr = list(corrMatrix.index)

print(corrMatrix)
print("The 10 selected features are:", selectedCorr)

SdssC      -0.493378
nHDon      -0.459505
nN_N       -0.447186
F02_CN     -0.426871
SM6_B      -0.418151
LOC        -0.411746
F03        -0.391805
Psi_i_1d   -0.381357
F03_CO     -0.354265
SpMax_L    -0.344159
Name: 0, dtype: float64
The 10 selected features are: ['SdssC', 'nHDon', 'nN_N', 'F02_CN', 'SM6_B', 'LOC', 'F03', 'Psi_i_1d', 'F03_CO', 'SpMax_L']


Random Forest Feature Selection

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


sel = SelectFromModel(RandomForestClassifier(), max_features=10)
sel.fit(X_train_processed, y_train)

selectedForest = []
selectedForestIndixes = []

for feature_list_index in sel.get_support(indices=True):
    selectedForestIndixes.append(feature_list_index)
    selectedForest.append(X_cols[feature_list_index])
    print("Feature: ", X_cols[feature_list_index], "| Importance: ", sel.estimator_.feature_importances_[feature_list_index])

Feature:  SpMax_L | Importance:  0.06690662116058628
Feature:  C | Importance:  0.051812353914987196
Feature:  F03 | Importance:  0.050345870873060146
Feature:  SdssC | Importance:  0.08674208590421663
Feature:  LOC | Importance:  0.052008967071320064
Feature:  SM6_L | Importance:  0.0524074184687339
Feature:  F03_CO | Importance:  0.035343463030889824
Feature:  nN_N | Importance:  0.05649352387372817
Feature:  nHDon | Importance:  0.06283164567733486
Feature:  nX | Importance:  0.041586306372976695


StepWise (Forward)

In [9]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeClassifier

N,M = X_train_processed.shape

dtc = DecisionTreeClassifier()
sfs = SequentialFeatureSelector(dtc, n_features_to_select=10, direction="forward")

sfs.fit(X_train_processed, y_train)

features = sfs.get_support()
ST_for_selected_features = np.arange(M)[features]

print(X_cols[ST_for_selected_features])

Index(['SdssC', 'LOC', 'SM6_L', 'F03_CO', 'nArNO2', 'B03', 'C_026', 'F02_CN',
       'nHDon', 'nX'],
      dtype='object')


StepWise (BackWard)

In [10]:
dtc = DecisionTreeClassifier()
sfs = SequentialFeatureSelector(dtc, n_features_to_select=10, direction="backward")

sfs.fit(X_train_processed, y_train)

features = sfs.get_support()
ST_back_selected_features = np.arange(M)[features]

print(X_cols[ST_back_selected_features])

Index(['F01', 'NssssC', 'C', 'F03', 'SdssC', 'LOC', 'SM6_L', 'F03_CO', 'TI2_L',
       'nHDon'],
      dtype='object')


Indexes of Features Selcted

In [11]:
corr_index = []
rand_forest_index = selectedForestIndixes
step_for_index = list(ST_for_selected_features)
step_back_index = list(ST_back_selected_features)

count = 0
for col in X_cols:
    if col in selectedCorr:
        corr_index.append(count)
    count += 1

print(corr_index)
print(rand_forest_index)
print(step_for_index)
print(step_back_index)

[0, 10, 11, 13, 15, 18, 27, 33, 34, 38]
[0, 7, 10, 11, 13, 14, 15, 18, 34, 40]
[11, 13, 14, 15, 19, 24, 32, 33, 34, 40]
[3, 5, 7, 10, 11, 13, 14, 15, 30, 34]


In [12]:
selec_feat_indexes = [corr_index,
                      rand_forest_index,
                      step_for_index,
                      step_back_index, 
                      range(0, 41)]

selec_feat_names = ["Correlation",
                      "Random Forest",
                      "StepWise For",
                      "StepWise Back", 
                      "No Feature Selection"]

Importar metricas de classificação

In [13]:
from sklearn.metrics import f1_score, matthews_corrcoef, confusion_matrix, accuracy_score

## KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import sys


def gaussian(dsts):
    kernel_width = .5
    weights = np.exp(-(dsts**2)/kernel_width)
    return weights

for i in range(5):
    # print(i)
    print("Feature selection method: " + selec_feat_names[i])

    bestMCC = sys.float_info.min
    bestMCCParams = {
            "neighbors": 0,
            "weigths": 0,
            "f1Score": 0
    }

    X_train = X_train_processed[:, selec_feat_indexes[i]]
    X_test = X_test_processed[:, selec_feat_indexes[i]]

    for k in range(2, 20, 2):
        for wei in [gaussian, "distance", "uniform"]:
            knn = KNeighborsClassifier(n_neighbors=k, weights=wei)
            knn.fit(X_train, y_train)
            preds = knn.predict(X_test)
            mcc = matthews_corrcoef(y_test, preds)
            if(mcc > bestMCC):
                bestMCC = mcc
                bestMCCParams["neighbors"] = k
                bestMCCParams["weigths"] = wei
                bestMCCParams["f1Score"] = f1_score(y_test, preds)

    print(f"The best MCC we found was {'{:.4f}'.format(bestMCC)} with the hyperparameters {bestMCCParams}")


# FIT BEST KNN MODEL
X_train = X_train_processed[:, step_back_index]
X_test = X_test_processed[:, step_back_index]

knn = KNeighborsClassifier(n_neighbors=bestMCCParams["neighbors"], weights=bestMCCParams["weigths"])
knn.fit(X_train, y_train)
preds = knn.predict(X_test)

bestKNNCM = confusion_matrix(y_test, preds)

print("\nConfusion Matrix of best KNN model\n", bestKNNCM)

            
# plt.plot(range(2, 20, 2), f1gauss, label="gaussian")
# plt.plot(range(2, 20, 2), f1distance, label="distance")
# plt.plot(range(2, 20, 2), f1noWei, label="no weights")
# plt.legend()
# plt.grid()
# plt.show()

Feature selection method: Correlation
The best MCC we found was 0.7719 with the hyperparameters {'neighbors': 4, 'weigths': <function gaussian at 0x000001B313DF9F70>, 'f1Score': 0.9656377402446127}
Feature selection method: Random Forest
The best MCC we found was 0.7711 with the hyperparameters {'neighbors': 12, 'weigths': 'distance', 'f1Score': 0.9665898617511521}
Feature selection method: StepWise For
The best MCC we found was 0.7863 with the hyperparameters {'neighbors': 8, 'weigths': <function gaussian at 0x000001B313DF9F70>, 'f1Score': 0.9680418361417781}
Feature selection method: StepWise Back
The best MCC we found was 0.8135 with the hyperparameters {'neighbors': 4, 'weigths': 'distance', 'f1Score': 0.9721577726218098}
Feature selection method: No Feature Selection
The best MCC we found was 0.6470 with the hyperparameters {'neighbors': 2, 'weigths': 'uniform', 'f1Score': 0.9477351916376306}

Confusion Matrix of best KNN model
 [[131  29]
 [ 31 818]]


## Random Forest

In [15]:
for i in range(5):
        bestMCC = sys.float_info.min
        bestMCCParams = {
                "max_depth": 0,
                "min_impurity_decrease": 0,
                "f1Score": 0
        }

        print("Feature selection method: " + selec_feat_names[i])

        X_train = X_train_processed[:, selec_feat_indexes[i]]
        X_test = X_test_processed[:, selec_feat_indexes[i]]


        for md in range(2, 20, 2):
                for mid in np.arange(0.05, 0.2, 0.05):
                        rf = RandomForestClassifier(n_estimators=50, criterion="entropy", max_depth=md, min_impurity_decrease=mid)
                        rf.fit(X_train, y_train)
                        preds = rf.predict(X_test)
                        f1Score = f1_score(y_test, preds) 
                        mcc = matthews_corrcoef(y_test, preds) 
                        
                        if(mcc > bestMCC):
                                bestMCC = mcc
                                bestMCCParams["max_depth"]= md
                                bestMCCParams["min_impurity_decrease"] = mid
                                bestMCCParams["f1Score"] = f1Score
                                
        
        print(f"The best MCC we found was {'{:.4f}'.format(bestMCC)} with the hyperparameters {bestMCCParams}")


# FIT BEST RF MODEL
X_train = X_train_processed[:, rand_forest_index]
X_test = X_test_processed[:, rand_forest_index]

rf = RandomForestClassifier(n_estimators=50, criterion="entropy", max_depth=bestMCCParams["max_depth"], min_impurity_decrease=bestMCCParams["min_impurity_decrease"])
rf.fit(X_train, y_train)
preds = rf.predict(X_test)

bestRFCM = confusion_matrix(y_test, preds)
print("\nConfusion Matrix of best RF model\n", bestRFCM)

Feature selection method: Correlation
The best MCC we found was 0.6309 with the hyperparameters {'max_depth': 6, 'min_impurity_decrease': 0.05, 'f1Score': 0.9477311889718552}
Feature selection method: Random Forest
The best MCC we found was 0.7084 with the hyperparameters {'max_depth': 4, 'min_impurity_decrease': 0.05, 'f1Score': 0.9556074766355139}
Feature selection method: StepWise For
The best MCC we found was 0.6619 with the hyperparameters {'max_depth': 14, 'min_impurity_decrease': 0.05, 'f1Score': 0.9516685845799769}
Feature selection method: StepWise Back
The best MCC we found was 0.6518 with the hyperparameters {'max_depth': 10, 'min_impurity_decrease': 0.05, 'f1Score': 0.9515116942384484}
Feature selection method: No Feature Selection
The best MCC we found was 0.6134 with the hyperparameters {'max_depth': 14, 'min_impurity_decrease': 0.05, 'f1Score': 0.9480156512017887}

Confusion Matrix of best RF model
 [[100  60]
 [ 25 824]]


## Bagging Classifier

In [16]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier, BaggingRegressor

for i in range(5):
    print("Feature selection method: " + selec_feat_names[i])

    X_train = X_train_processed[:, selec_feat_indexes[i]]
    X_test = X_test_processed[:, selec_feat_indexes[i]]

    bestMCC = sys.float_info.min
    bestMCCParams = {
                "n_estimators": 0,
                "f1Score": 0
        }

    for n in range(2, 10, 2):
        bag = BaggingClassifier(base_estimator=GaussianNB(), n_estimators=n, random_state=22)
        bag.fit(X_train, y_train)
        preds = bag.predict(X_test)
        mcc = matthews_corrcoef(y_test, preds)

        if(bestMCC < mcc):
            bestMCC = mcc
            bestMCCParams["n_estimators"] = n
            bestMCCParams["f1Score"] =  f1_score(y_test, preds)


    print(f"The best MCC we found was {'{:.4f}'.format(bestMCC)} with the hyperparameters {bestMCCParams}")


# FIT BEST Bagging MODEL
X_train = X_train_processed[:, step_for_index]
X_test = X_test_processed[:, step_for_index]

bag = BaggingClassifier(base_estimator=GaussianNB(), n_estimators=bestMCCParams["n_estimators"], random_state=22)
bag.fit(X_train, y_train)
preds = bag.predict(X_test)

bestBGCM = confusion_matrix(y_test, preds)
print("\nConfusion Matrix of best Bagging model\n", bestBGCM)


Feature selection method: Correlation
The best MCC we found was 0.5872 with the hyperparameters {'n_estimators': 6, 'f1Score': 0.9317507418397627}
Feature selection method: Random Forest
The best MCC we found was 0.6984 with the hyperparameters {'n_estimators': 8, 'f1Score': 0.9523248969982343}
Feature selection method: StepWise For
The best MCC we found was 0.7452 with the hyperparameters {'n_estimators': 4, 'f1Score': 0.9632606199770379}
Feature selection method: StepWise Back
The best MCC we found was 0.6945 with the hyperparameters {'n_estimators': 2, 'f1Score': 0.9502958579881656}
Feature selection method: No Feature Selection
The best MCC we found was 0.7635 with the hyperparameters {'n_estimators': 4, 'f1Score': 0.9653579676674365}

Confusion Matrix of best Bagging model
 [[106  54]
 [ 10 839]]


## Fitting and validating the best model 

(KNN | Step Back | Gaussian | k=2)

In [17]:
# X_train = X_train_processed[:, step_back_index]
# X_test = X_test_processed[:, step_back_index]

scaler = StandardScaler()
scaler.fit(X_train_IVS)

X_train_IVS_scaled = scaler.transform(X_train_IVS)
X_IVS_scaled = scaler.transform(X_IVS)

imputerMean = SimpleImputer(missing_values = np.nan, strategy = "mean") 
imputerMedian = SimpleImputer(missing_values = np.nan, strategy = "median") 

ct = ColumnTransformer(
    [('mean_imputer', imputerMean, useMean),
     ('median_imputer', imputerMedian, useMedian)]
)

ct.fit(X_train_IVS_scaled)

X_train_IVS_processed = ct.transform(X_train_IVS_scaled)
X_IVS_processed = ct.transform(X_IVS_scaled)


knn = KNeighborsClassifier(n_neighbors=2, weights=gaussian)
knn.fit(X_train_IVS_processed, y_train_IVS)

preds = knn.predict(X_IVS_processed)

mcc = matthews_corrcoef(y_IVS, preds)
cm = confusion_matrix(y_IVS, preds)

print("Our Selected Model is a KNN, where k=2, using features selected with a Backwards StepWise and a Gaussian weight function: \n")
print("The MCC is: ", mcc)
print("The Confusion Matrix is: \n", cm)

Our Selected Model is a KNN, where k=2, using features selected with a Backwards StepWise and a Gaussian weight function: 

The MCC is:  0.8483190422946583
The Confusion Matrix is: 
 [[ 208   46]
 [  16 1237]]
